In [ ]:
import $ivy.`org.apache.spark::spark-sql:3.5.5`

In [ ]:
// neo4j-spark-connector
/*
import coursierapi._
interp.repositories() ++= Seq(MavenRepository.of("https://repos.spark-packages.org/"))
interp.load.ivy(("neo4j" % "neo4j-spark-connector" % "5.3.3-s_2.12"))
*/

## Spark Session с подключением к Neo4j

In [ ]:
import org.apache.spark.sql._

In [ ]:
val spark = SparkSession
                .builder()
                .master("local[*]")
                .appName("SparkNeo4j")
                .config("spark.log.level", "WARN")
                .config("spark.jars.packages", "neo4j:neo4j-spark-connector:5.3.3-s_2.12")
                .config("neo4j.url", "neo4j://localhost:7687")
                .config("neo4j.authentication.type", "basic")
                .config("neo4j.authentication.basic.username", "neo4j")
                .config("neo4j.authentication.basic.password", "password")
                .getOrCreate()

import spark.implicits._

## Read data from Neo4j into Spark

### Movies

In [ ]:
val movies = spark.read
                .format("org.neo4j.spark.DataSource")
                .option("labels", ":Movie")
                .load()

In [ ]:
movies.printSchema()

In [ ]:
movies.show(10, false)

### Person

In [ ]:
val person = spark.read
                .format("org.neo4j.spark.DataSource")
                .option("labels", ":Person")
                .load()

In [ ]:
person.printSchema()

In [ ]:
person.show(10, false)

### ACTED_IN

In [ ]:
val actedin = spark.read
                .format("org.neo4j.spark.DataSource")
                .option("relationship", "ACTED_IN")
                .option("relationship.source.labels", ":Person")
                .option("relationship.target.labels", ":Movie")
                .load()

In [ ]:
actedin.printSchema()

In [ ]:
actedin.show(10, false)

### DataFrame with nodes as map

In [ ]:
val actedinMap = spark.read
                    .format("org.neo4j.spark.DataSource")
                    .option("relationship.nodes.map", true)
                    .option("relationship", "ACTED_IN")
                    .option("relationship.source.labels", ":Person")
                    .option("relationship.target.labels", ":Movie")
                    .load()

In [ ]:
actedinMap.printSchema()

In [ ]:
actedinMap.show(10, false)

### Directed

In [ ]:
val directed = spark.read
                .format("org.neo4j.spark.DataSource")
                .option("relationship", "DIRECTED")
                .option("relationship.source.labels", ":Person")
                .option("relationship.target.labels", ":Movie")
                .load()

In [ ]:
directed.printSchema()

In [ ]:
directed.show(10, false)

### Read arbitrary data via Cypher query

In [ ]:
val cypher = spark.read
                .format("org.neo4j.spark.DataSource")
                .option("query", """
                    // Extend Tom Hanks co-actors, to find co-co-actors who haven't worked with Tom Hanks
                    MATCH (tom:Person {name:"Tom Hanks"})-[:ACTED_IN]->(m)<-[:ACTED_IN]-(coActors),
                    (coActors)-[:ACTED_IN]->(m2)<-[:ACTED_IN]-(cocoActors)
                    WHERE NOT (tom)-[:ACTED_IN]->()<-[:ACTED_IN]-(cocoActors)
                        AND tom <> cocoActors
                    RETURN cocoActors.name AS Recommended, count(*) AS Strength
                    ORDER BY Strength DESC
                    """)
                .load()

In [ ]:
cypher.printSchema()

In [ ]:
cypher.show(10, false)

### Return all the actors that have also directed a movie.

In [ ]:
val actorsDirectors = spark.read
                        .format("org.neo4j.spark.DataSource")
                        .option("query", """
                                MATCH (p:Person)
                                MATCH (p)-[:ACTED_IN]->(m:Movie)
                                MATCH (p)-[:DIRECTED]->(m1:Movie)
                                RETURN p.name AS name, collect(m.title) AS acted_in, collect(m1.title) AS directed
                                """)
                        .load()

In [ ]:
actorsDirectors.printSchema()

In [ ]:
actorsDirectors.show(10, false)

## Write data from Spark to Neo4j

In [ ]:
val products = spark.read
                .format("csv")
                .option("inferSchema", true)
                .load("desktop-csv-import/products.csv")
                .withColumnsRenamed(Map("_c0" -> "id", "_c1" -> "name", "_c2" -> "price"))

In [ ]:
products.printSchema()

In [ ]:
products.show(10, false)

In [ ]:
products.count()

In [ ]:
val orders = spark.read
                .format("csv")
                .option("inferSchema", true)
                .option("header", true)
                .load("desktop-csv-import/orders.csv")
                .selectExpr("orderID AS id", "CAST(orderDate AS TIMESTAMP) AS date", "shipCountry")

In [ ]:
orders.printSchema()

In [ ]:
orders.show(10, false)

In [ ]:
orders.count()

### Write nodes via label option

In [ ]:
products.write
        .format("org.neo4j.spark.DataSource")
        .mode("append")
        .option("labels", ":Product")
        .save()

In [ ]:
orders.write
      .format("org.neo4j.spark.DataSource")
      .mode("overwrite")
      .option("labels", ":Order")
      .option("schema.optimization.type", "NODE_CONSTRAINTS")
// this is necessary in order to specify what is the constraint field
      .option("node.keys", "id")
      .save()

### Write relationships via relationship option

In [ ]:
val orderDetails = spark.read
                    .format("csv")
                    .option("inferSchema", true)
                    .option("header", true)
                    .load("desktop-csv-import/order-details.csv")

In [ ]:
orderDetails.printSchema()

In [ ]:
orderDetails.show(10, false)

In [ ]:
orderDetails.count()

In [ ]:
orderDetails.write
            .format("org.neo4j.spark.DataSource")
            .mode("overwrite")
            .option("relationship", "CONTAINS")
            .option("relationship.save.strategy", "keys")
            .option("relationship.source.labels", ":Product")
            .option("relationship.source.save.mode", "Match")
            .option("relationship.source.node.keys", "productID:id")
            .option("relationship.target.labels", ":Order")
            .option("relationship.target.save.mode", "Match")
            .option("relationship.target.node.keys", "orderID:id")
            .option("relationship.properties", "quantity:quantityOrdered")
            .save()

### Write custom graphs via Cypher Query

In [ ]:
val actorOrders = Seq(
  ("Cuba Gooding Jr.", 1, Array(11, 42, 72), Array(1, 2, 3), "2022-06-07 00:00:00"),
  ("Tom Hanks", 2, Array(24, 55, 75), Array(3, 2, 1), "2022-06-06 00:00:00")
)

In [ ]:
val actorOrdersDF = spark
                    .createDataFrame(actorOrders)
                    .withColumnsRenamed(
                        Map(
                            "_1" -> "actor_name",
                            "_2" -> "order_id",
                            "_3" -> "products",
                            "_4" -> "quantities",
                            "_5" -> "order_date"))

In [ ]:
actorOrdersDF.printSchema()

In [ ]:
actorOrdersDF.show(10, false)

In [ ]:
actorOrdersDF.write
             .format("org.neo4j.spark.DataSource")
             .mode("overwrite")
             .option("query", """
                     MATCH (person:Person {name: event.actor_name})
                     MERGE (order:Order {id: event.order_id, date: datetime(replace(event.order_date, ' ', 'T'))})
                     MERGE (person)-[:CREATED]->(order)
                     WITH event, order
                     UNWIND range(0, size(event.products) - 1) AS index
                     MATCH (product:Product {id: event.products[index]})
                     MERGE (product)-[:CONTAINS{quantityOrdered: event.quantities[index]}]->(order)
                    """)
             .save()